In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#### delete from here

In [2]:
#sizes = [0.5, 0.1, 0.05, 0.01]
#seeds = [0, 21, 63, 84]
#steps = [5500000, 6000000]
#for seed in seeds:
#    for step in steps:
#        random.seed(seed)
#        np.random.seed(seed)
#        os.environ['PYTHONHASHSEED']=str(seed)
#        torch.manual_seed(seed)
#        torch.use_deterministic_algorithms(True)
#        #train_df = pd.read_csv(f'../../final/data/train_set_basic_{size}_seed_{seed}.csv')
#        train_df = pd.read_csv(f'../../final/data/train_set_basic.csv')
#        train_df = train_df.fillna(-1)
#        X_train = train_df.iloc[:, 0:-1]
#        y_train = train_df.iloc[:, -1]
#        X_train, y_train = np.array(X_train), np.array(y_train)
#        dqn_model = utils.stable_dqn3(X_train, y_train, step, True, 
#                                      f'../../final/models/dqn_basic_seed_{seed}_{step}')

#### end here

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [4]:
train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.401651,-1.000000,-1.000000,0.000000,116.415615,103.966653,-1.000000,2.712885,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.0
1,7.029873,102.717562,2.079564,-1.000000,-1.000000,79.373697,212.817785,2.657004,1,0.675576,104.688780,50.452242,-1.000000,-1.000000,49.832026,21.089620,-1.000000,3.0
2,15.900836,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,0.999932,90.916733,60.845387,77.678965,1.513323,139.638756,47.702509,-1.000000,0.0
3,12.710083,437.896009,-1.000000,-1.000000,392.155888,103.338649,91.089902,3.689834,1,1.431816,31.576192,124.321572,79.407308,24.911028,84.817826,-1.000000,-1.000000,7.0
4,9.256332,267.393719,-1.000000,5.758205,258.911569,76.491274,189.160039,-1.000000,1,1.259486,-1.000000,-1.000000,55.681222,-1.000000,-1.000000,27.768995,73.059709,3.0


In [5]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [6]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [7]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [8]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

#### Training 

In [9]:
#for steps in [int(24e6), ]:
#    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_4_missing_3_{steps}')
for steps in [int(21.3e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_6_missing_3_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.89     |
|    ep_rew_mean      | -0.78    |
|    exploration_rate | 0.877    |
|    success_rate     | 0.11     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 1111     |
|    time_elapsed     | 249      |
|    total_timesteps  | 276880   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.74     |
|    n_updates        | 56719    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.91     |
|    ep_rew_mean      | -0.82    |
|    exploration_rate | 0.743    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 1108     |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.38     |
|    ep_rew_mean      | -0.34    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.36     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 1045     |
|    time_elapsed     | 4664     |
|    total_timesteps  | 4874641  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.167    |
|    n_updates        | 1206160  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.58     |
|    ep_rew_mean      | -0.52    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.28     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 1036     |
|    time_elapsed     | 5040     |
|    total_timesteps  | 5223551  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.4      |
|    ep_rew_mean      | -0.3     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.41     |
| time/               |          |
|    episodes         | 3100000  |
|    fps              | 969      |
|    time_elapsed     | 10417    |
|    total_timesteps  | 10100800 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.199    |
|    n_updates        | 2512699  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.79     |
|    ep_rew_mean      | -0.28    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.37     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 957      |
|    time_elapsed     | 10917    |
|    total_timesteps  | 10455125 |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.87     |
|    ep_rew_mean      | -0.44    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.29     |
| time/               |          |
|    episodes         | 4600000  |
|    fps              | 903      |
|    time_elapsed     | 17126    |
|    total_timesteps  | 15473562 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0777   |
|    n_updates        | 3855890  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.09     |
|    ep_rew_mean      | -0.22    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.42     |
| time/               |          |
|    episodes         | 4700000  |
|    fps              | 904      |
|    time_elapsed     | 17487    |
|    total_timesteps  | 15823057 |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.47     |
|    ep_rew_mean      | -0.48    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.32     |
| time/               |          |
|    episodes         | 6100000  |
|    fps              | 916      |
|    time_elapsed     | 22647    |
|    total_timesteps  | 20759017 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.177    |
|    n_updates        | 5177254  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.3      |
|    ep_rew_mean      | -0.36    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.35     |
| time/               |          |
|    episodes         | 6200000  |
|    fps              | 918      |
|    time_elapsed     | 22972    |
|    total_timesteps  | 21101831 |
| train/              |          |
|    learning_rate  

In [ ]:
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

#### Testing

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_16000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
test_df[test_df.y_pred==4]

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_9000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])

In [ ]:
a = [94.679, 96.764, 90.942, 97.493, 96.064]
np.mean(a), np.std(a)

In [ ]:
b = [90.986, 95.021, 93.829, 92.200, 95.607]
np.mean(b), np.std(b)

In [ ]:
c =  [94.657, 82.229, 91.550, 95.179, 95.150]
np.mean(c), np.std(c)